<a href="https://colab.research.google.com/github/Daisuke134/MW_detection/blob/main/MW_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 複数のチャンクサイズで試す+スレッド並列処理

import mne
import time
from concurrent.futures import ThreadPoolExecutor

def load_data(fname, chunk_size):
    raw = mne.io.read_raw_bdf(fname, preload=False)
    total_time = 0
    for start in range(0, raw.n_times, chunk_size):
        start_time = time.time()
        data_chunk = raw.get_data(start=start, stop=start+chunk_size)
        total_time += time.time() - start_time
    return total_time

bdf_files = [
    "/content/drive/MyDrive/BDF/sub-01_ses-1_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-2_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-3_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-4_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-5_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-6_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-7_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-8_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-9_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-10_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-11_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-1_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-2_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-3_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-4_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-5_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-6_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-7_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-8_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-9_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-10_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-11_task-BreathCounting_eeg.bdf",
]  # ファイルリスト

# 複数のチャンクサイズで試す
chunk_sizes = [50000, 100000, 150000, 200000, 250000, 300000, 500000, 1000000]
best_time = float('inf')
best_chunk_size = None

for chunk_size in chunk_sizes:
    with ThreadPoolExecutor() as executor:
        times = list(executor.map(lambda fname: load_data(fname, chunk_size), bdf_files))
    avg_time = sum(times) / len(times)
    if avg_time < best_time:
        best_time = avg_time
        best_chunk_size = chunk_size

print(f"最適なチャンクサイズ: {best_chunk_size}, 平均読み込み時間: {best_time}")

In [ ]:
#スレッドによる並列化＋タスクの分割

import mne
import time
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor

def load_data_chunk(fname, start, stop):
    raw = mne.io.read_raw_bdf(fname, preload=False)
    data_chunk = raw.get_data(start=start, stop=stop)
    return data_chunk

bdf_files = [
    "/content/drive/MyDrive/BDF/sub-01_ses-1_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-2_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-3_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-4_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-5_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-6_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-7_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-8_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-9_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-10_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-11_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-1_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-2_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-3_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-4_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-5_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-6_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-7_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-8_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-9_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-10_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-11_task-BreathCounting_eeg.bdf",
]

chunk_size = 1000000  # チャンクサイズの設定
start_time = time.time()

with ThreadPoolExecutor() as executor:
    future_to_chunk = {executor.submit(load_data_chunk, fname, start, start+chunk_size): fname for fname in bdf_files for start in range(0, raw.n_times, chunk_size)}
    for future in concurrent.futures.as_completed(future_to_chunk):
        fname = future_to_chunk[future]
        try:
            data_chunk = future.result()
        except Exception as exc:
            print(f'{fname} generated an exception: {exc}')

elapsed_time = time.time() - start_time
print(f"ファイル読み込みにかかった時間: {elapsed_time:.2f} 秒")

In [ ]:
#マルチプロセス

from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import mne
import time

def process_file(fname, n_times):
    def load_data_chunk(start, stop):
        raw = mne.io.read_raw_bdf(fname, preload=False)
        data_chunk = raw.get_data(start=start, stop=stop)
        return data_chunk

    chunk_size = 1000000
    with ThreadPoolExecutor() as executor:
        start_time = time.time()
        chunks = [executor.submit(load_data_chunk, start, start+chunk_size) for start in range(0, n_times, chunk_size)]
        results = [future.result() for future in concurrent.futures.as_completed(chunks)]
        total_time = time.time() - start_time
    return total_time

def execute_process_file(args):
    return process_file(*args)

bdf_files = [
    "/content/drive/MyDrive/BDF/sub-01_ses-1_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-2_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-3_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-4_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-5_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-6_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-7_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-8_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-9_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-10_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-11_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-1_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-2_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-3_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-4_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-5_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-6_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-7_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-8_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-9_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-10_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-11_task-BreathCounting_eeg.bdf",
]
#(fname)とn_timesのリストを取得
file_n_times = [(fname, mne.io.read_raw_bdf(fname, preload=False).n_times) for fname in bdf_files]

with ProcessPoolExecutor() as executor:
    times = list(executor.map(execute_process_file, file_n_times))

avg_time = sum(times) / len(times)
print(f"平均読み込み時間: {avg_time:.2f} 秒")

In [ ]:
#ヒートマップ作成
# 必要なライブラリをインストール
!pip install mne matplotlib

# ライブラリをインポート
import mne
import numpy as np
import matplotlib.pyplot as plt

# ファイル名リストの例 (適宜修正してください)
bdf_files = [
    "/content/drive/MyDrive/BDF/sub-01_ses-1_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-2_task-BreathCounting_eeg.bdf",
]

# ヒートマップを作成する関数
def create_heatmap(data, start, end, sfreq, event_index):
    start_sample = int(event_index + start * sfreq)
    end_sample = int(event_index + end * sfreq)
    event_data = data[:64, start_sample:end_sample]
    plt.imshow(event_data, aspect='auto', extent=[start, end, event_data.shape[0], 0], cmap='jet')
    plt.colorbar()
    plt.xlim(-5, 5)
    plt.xlabel('Time (s)')
    plt.ylabel('Channels')
    plt.ylim(event_data.shape[0]-1, 0)
    plt.title('EEG Data Heatmap')
    plt.show()

# データ選択とヒートマップ作成
for fname in bdf_files:
    raw = mne.io.read_raw_bdf(fname, preload=True)
    events = mne.find_events(raw, initial_event=True)
    sfreq = raw.info['sfreq']  # サンプリングレートの取得
    data = raw.get_data()

    # Mind-wandering (MW)の最初のイベントを見つけてヒートマップを作成
    mw_events = [event[0] for event in events if event[2] == 30]

    if mw_events:
        create_heatmap(data, -5, 5, sfreq, mw_events[0])  # 最初のMWイベントのヒートマップを作成

    # 各イベントインデックスについてヒートマップを生成します。
   #for index in mw_events:
    # ヒートマップを生成する関数を呼び出します。
        #create_heatmap(data, -5, 5, sfreq, index)

        # 追加のMWイベントのヒートマップ（オプション）
        #for i, event in enumerate(mw_events[1:], start=1):  # 最初のイベントをスキップ
            #create_heatmap(data, -5, 5, sfreq, event)  # 次のMWイベントのヒートマップを作成

# 最初の列がチャンネル名を含んでいるか確認
#ch_names = raw.info['ch_names']
#print('The first column contains channel names:', ch_names[-5:])

#print(len(mw_events))

In [ ]:
#スレッドによる並列化＋タスクの分割(新しい)

import mne
import time
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor

def load_data_chunk(fname, start, stop):
    #fnameファイルを開き、各チャンクのデータ読み込み
    raw = mne.io.read_raw_bdf(fname, preload=False)
    data_chunk = raw.get_data(start=start, stop=stop)
    return data_chunk

bdf_files = [
    "/content/drive/MyDrive/BDF/sub-01_ses-1_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-2_task-BreathCounting_eeg.bdf",

]

chunk_size = 1000000  # チャンクサイズの設定
start_time = time.time()

with ThreadPoolExecutor() as executor:
    future_to_chunk = {}
    for fname in bdf_files:
        #どのようにチャンクに分割するか決めるために、n_timesを取得する
        raw = mne.io.read_raw_bdf(fname, preload=False)
        for start in range(0, raw.n_times, chunk_size):
            future = executor.submit(load_data_chunk, fname, start, start+chunk_size)
            future_to_chunk[future] = fname

    for future in concurrent.futures.as_completed(future_to_chunk):
        fname = future_to_chunk[future]
        try:
            data_chunk = future.result()
        except Exception as exc:
            print(f'{fname} generated an exception: {exc}')

elapsed_time = time.time() - start_time
print(f"ファイル読み込みにかかった時間: {elapsed_time:.2f} 秒")

In [ ]:
#異なる時間窓によるデータ分割(これを高速化する！！)
import numpy as np
import mne

bdf_files = [
    "/content/drive/MyDrive/BDF/sub-01_ses-1_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-2_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-3_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-4_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-5_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-6_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-7_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-8_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-9_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-10_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-11_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-1_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-2_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-3_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-4_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-5_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-6_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-7_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-8_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-9_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-10_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-11_task-BreathCounting_eeg.bdf",
]  # ファイル名リスト

def extract_data_segments(raw_data, events, event_id, start_offset, end_offset):
    event_indices = [event[0] for event in events if event[2] == event_id]
    segments = []
    for index in event_indices:
        segment = raw_data[:, index+start_offset:index+end_offset].copy()
        segments.append(segment)
    return segments

sampling_rate = 1024  # サンプリングレート

# MWとFの時間窓を定義
mw_time_ranges = [
    (-10 * sampling_rate, -2 * sampling_rate),
]
f_time_ranges = [
    (0, 8 * sampling_rate),
]

for fname in bdf_files:
    raw = mne.io.read_raw_bdf(fname, preload=True)
    events = mne.find_events(raw, initial_event=True)
    data = raw._data[0:64]

    for start, end in mw_time_ranges:
        mw_segments = extract_data_segments(data, events, 30, start, end)
        for i, segment in enumerate(mw_segments):
            np.savetxt(f"MW_{i}.txt", segment, delimiter=',', header='MW')

    for start, end in f_time_ranges:
        f_segments = extract_data_segments(data, events, 50, start, end)
        for i, segment in enumerate(f_segments):
            np.savetxt(f"F_{i}.txt", segment, delimiter=',', header='F')

In [ ]:
#並列化処理+データ抽出
!pip install mne
import numpy as np
import mne
import time
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor

bdf_files = [
    "/content/drive/MyDrive/BDF/sub-01_ses-1_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-2_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-3_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-4_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-5_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-6_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-7_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-8_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-9_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-10_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-11_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-1_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-2_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-3_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-4_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-5_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-6_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-7_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-8_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-9_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-10_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-11_task-BreathCounting_eeg.bdf",
]  # ファイル名リスト

def extract_data_segments(raw_data, events, event_id, start_offset, end_offset):
    event_indices = [event[0] for event in events if event[2] == event_id]
    segments = []
    for index in event_indices:
        segment = raw_data[:, index+start_offset:index+end_offset].copy()
        segments.append(segment)
    return segments

def load_data_chunk(fname, start, stop):
    raw = mne.io.read_raw_bdf(fname, preload=False)  # データをディスクに取得
    data_chunk = raw.get_data(start=start, stop=stop)  # 指定範囲のデータを取得
    events = mne.find_events(raw, initial_event=True)  # イベントを抽出
    return data_chunk, events

def process_file(fname, mw_time_ranges, f_time_ranges):
    raw = mne.io.read_raw_bdf(fname, preload=True)
    events = mne.find_events(raw, initial_event=True)
    data = raw._data[0:64]
    mw_count = 0  # MWファイルのカウント
    f_count = 0  # Fファイルのカウント

    for start, end in mw_time_ranges:
        mw_segments = extract_data_segments(data, events, 30, start, end)
        for i, segment in enumerate(mw_segments):
            np.savetxt(f"MW_{i}.txt", segment, delimiter=',', header='MW')
            mw_count += 1  # MWファイルのカウントを増やす

    for start, end in f_time_ranges:
        f_segments = extract_data_segments(data, events, 50, start, end)
        for i, segment in enumerate(f_segments):
            np.savetxt(f"F_{i}.txt", segment, delimiter=',', header='F')
            f_count += 1  # Fファイルのカウントを増やす
    return mw_count, f_count

chunk_size = 500000
#start_time = time.time()

sampling_rate = 1024  # サンプリングレート

# MWとFの時間窓を定義
mw_time_ranges = [
    (-10 * sampling_rate, -2 * sampling_rate),  # MWの時間窓
]
f_time_ranges = [
    (0, 8 * sampling_rate),  # Fの時間窓
]

mw_total_count = 0 #MWファイルのカウント
f_total_count = 0 #Fファイルのカウント

with ThreadPoolExecutor() as executor:
    file_n_times = [(fname, mne.io.read_raw_bdf(fname, preload=False).n_times) for fname in bdf_files]
    results = list(executor.map(lambda f: process_file(f[0], mw_time_ranges, f_time_ranges), file_n_times))

#elapsed_time = time.time() - start_time
#print(f"ファイル読み込みにかかった時間: {elapsed_time:.2f} 秒")

for mw_count, f_count in results:
    mw_total_count += mw_count
    f_total_count += f_count

print(f"Total MW files saved: {mw_total_count}")
print(f"Total F files saved: {f_total_count}")

In [ ]:
#txtのリストをシャッフルする
import pandas as pd
import random

# CSVファイルを読み込む
df = pd.read_csv("/content/A.csv")

# データフレームをシャッフルする
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# シャッフルされたデータフレームを新しいCSVとして保存する
df.to_csv("/content/A.csv", index=False)

df.head(5)

In [ ]:
#DATA SELECTION(EEGデータの選択)
!pip install mne
import mne
import numpy as np

bdf_files = [
    "/content/drive/MyDrive/BDF/sub-01_ses-1_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-2_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-3_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-4_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-5_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-6_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-7_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-8_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-9_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-10_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-01_ses-11_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-1_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-2_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-3_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-4_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-5_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-6_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-7_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-8_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-9_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-10_task-BreathCounting_eeg.bdf",
    "/content/drive/MyDrive/BDF/sub-02_ses-11_task-BreathCounting_eeg.bdf",
]

mw_count = 0
f_count = 0

for fname in bdf_files:
    raw = mne.io.read_raw_bdf(fname, preload=True)
    events = mne.find_events(raw, initial_event=True)
    info = raw.info
    data = raw._data[0:64]

    # Mind-wandering (MW)
    mw = [event[0] for event in events if event[2] == 30]
    shape_myarray = (64, data.shape[1])
    mw_duration = np.zeros(shape_myarray)

    for j in range(len(mw)):
        mw_duration = data[:, mw[j]-(10*1024):mw[j]-(2*1024)].copy()
        np.savetxt("MW_" + str(mw_count) + ".txt", mw_duration, delimiter=',', header='MW')
        mw_count += 1

    # Focusing (F)
    f = [event[0] for event in events if event[2] == 50]
    shape_myarray = (64, data.shape[1])
    f_duration = np.zeros(shape_myarray)

    for j in range(len(f)):
        f_duration = data[:, f[j]:(f[j]+(8*1024))].copy()
        np.savetxt("F_" + str(f_count) + ".txt", f_duration, delimiter=',', header='F')
        f_count += 1

print(f"Total F files saved: {f_count}")
print(f"Total MW files saved: {mw_count}")

In [ ]:
#Random Forest/SVMを使ったMW識別
import glob
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn.utils import shuffle
import scipy.signal
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold, cross_validate
from sklearn.model_selection import GridSearchCV
from scipy.signal import welch
from sklearn.ensemble import RandomForestClassifier
!pip install optuna
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

"""
def extract_feature(data):
  # 既存の特徴量
    mean_signal = np.mean(data, axis=1)
    std_signal = np.std(data, axis=1)
    max_signal = np.max(data, axis=1)
    min_signal = np.min(data, axis=1)
    # 新たな特徴量（パワースペクトル密度）
    freqs, power = welch(data, fs=1024, nperseg=1024)
    theta_band = np.mean(power[:, (freqs >= 4) & (freqs <= 8)], axis=1)
    alpha_band = np.mean(power[:, (freqs >= 8) & (freqs <= 12)], axis=1)
    beta_band = np.mean(power[:, (freqs >= 13) & (freqs <= 30)], axis=1)
    delta_band = np.mean(power[:, (freqs >= 1) & (freqs <= 4)], axis=1)
    return np.concatenate((mean_signal, std_signal, max_signal, min_signal, theta_band, alpha_band, beta_band, delta_band), axis=None)

def extract_feature(data):
    # 短い時間窓の特徴
    mean_signal_short = np.mean(data[:, :2048], axis=1)
    std_signal_short = np.std(data[:, :2048], axis=1)

    # 追加周波数帯域特徴
    freqs, power = welch(data, fs=1024, nperseg=1024)
    beta_band = np.mean(power[:, (freqs >= 13) & (freqs <= 30)], axis=1)
    delta_band = np.mean(power[:, (freqs >= 1) & (freqs <= 4)], axis=1)

    return np.concatenate((mean_signal_short, std_signal_short, beta_band, delta_band), axis=None)
"""
def extract_feature(data):
    # 既存の特徴量
    mean_signal = np.mean(data, axis=1)
    std_signal = np.std(data, axis=1)
    max_signal = np.max(data, axis=1)
    min_signal = np.min(data, axis=1)

    # 新たな特徴量（周波数帯域）
    freqs, power = welch(data, fs=1024, nperseg=1024)
    theta_band = np.mean(power[:, (freqs >= 4) & (freqs <= 8)], axis=1)
    alpha_band = np.mean(power[:, (freqs >= 8) & (freqs <= 12)], axis=1)
    beta_band = np.mean(power[:, (freqs >= 13) & (freqs <= 30)], axis=1)
    delta_band = np.mean(power[:, (freqs >= 1) & (freqs <= 4)], axis=1)

    # 2048分割
    mean_signal_8 = np.mean(data[:, :2048], axis=1)
    std_signal_8 = np.std(data[:, :2048], axis=1)
    mean_signal_9 = np.mean(data[:, 2048:4096], axis=1)
    std_signal_9 = np.std(data[:, 2048:4096], axis=1)
    mean_signal_10 = np.mean(data[:, 4096:6144], axis=1)
    std_signal_10 = np.std(data[:, 4096:6144], axis=1)
    mean_signal_11 = np.mean(data[:, 6144:], axis=1)
    std_signal_11 = np.std(data[:, 6144:], axis=1)

    # 半分に
    mean_signal_13 = np.mean(data[:, :4096], axis=1)
    std_signal_13 = np.std(data[:, :4096], axis=1)
    mean_signal_14 = np.mean(data[:, 4096:], axis=1)
    std_signal_14 = np.std(data[:, 4096:], axis=1)
    # 新たな特徴量（例：異なる時間範囲の特徴）
    #1024分割
    mean_signal_0 = np.mean(data[:, :1024], axis=1)
    std_signal_0 = np.std(data[:, :1024], axis=1)
    mean_signal_1 = np.mean(data[:, 1024:2048], axis=1)
    std_signal_1 = np.std(data[:, 1024:2048], axis=1)
    mean_signal_2 = np.mean(data[:, 2048:3072], axis=1)
    std_signal_2 = np.std(data[:, 2048:3072], axis=1)
    mean_signal_3 = np.mean(data[:, 3072:4096], axis=1)
    std_signal_3 = np.std(data[:, 3072:4096], axis=1)
    mean_signal_4 = np.mean(data[:, 4096:5120], axis=1)
    std_signal_4 = np.std(data[:, 4096:5120], axis=1)
    mean_signal_5 = np.mean(data[:, 5120:6144], axis=1)
    std_signal_5 = np.std(data[:, 5120:6144], axis=1)
    mean_signal_6 = np.mean(data[:, 6144:7168], axis=1)
    std_signal_6 = np.std(data[:, 6144:7168], axis=1)
    mean_signal_7 = np.mean(data[:, 7168:], axis=1)
    std_signal_7 = np.std(data[:, 7168:], axis=1)

    # 特徴量を結合
    return np.concatenate((mean_signal, std_signal, max_signal, min_signal, theta_band, alpha_band, beta_band, delta_band, mean_signal_0, std_signal_0, mean_signal_1, std_signal_1, mean_signal_2, std_signal_2, mean_signal_3, std_signal_3, mean_signal_4, std_signal_4,mean_signal_5, std_signal_5,mean_signal_6, std_signal_6,mean_signal_7, std_signal_7,mean_signal_8, std_signal_8,mean_signal_9, std_signal_9,mean_signal_10, std_signal_10,mean_signal_11, std_signal_11,mean_signal_13, std_signal_13,mean_signal_14, std_signal_14), axis=None)

# MWとFのファイルを取得
mw_files = glob.glob("/content/MW_*.txt")
f_files = glob.glob("/content/F_*.txt")

# 特徴とラベルのリストを作成
features = []
labels = []

# MWのデータから特徴を抽出
for fname in mw_files:
    data = np.loadtxt(fname, delimiter=',')
    feature = extract_feature(data)
    features.append(feature)
    #features.append(data)  # データを平坦化して追加
    labels.append(1)  # MWのラベル

# Fのデータから特徴を抽出
for fname in f_files:
    data = np.loadtxt(fname, delimiter=',')
    feature = extract_feature(data)
    features.append(feature)
    #features.append(data)  # データを平坦化して追加
    labels.append(0)  # Fのラベル

# 特徴とラベルをNumPy配列に変換
X = np.array(features)
y = np.array(labels)

# PCAによる次元削減
#pca = PCA(n_components=60)
#X_pca = pca.fit_transform(X)

# 追加: データのバランスを取るため、Fのサンプル数をMWの数に合わせる
#mw_sample_indices = np.where(y == 1)[0]  # MWのインデックス
#f_sample_indices = np.where(y == 0)[0]  # Fのインデックス
mw_sample_indices = np.random.choice(np.where(y == 1)[0], 250, replace=False)
f_sample_indices = np.random.choice(np.where(y == 0)[0], 250, replace=False)
#f_sample_indices = np.random.choice(f_sample_indices, len(mw_sample_indices), replace=False)  # ランダムに選択
f_sample_indices = f_sample_indices[:len(mw_sample_indices)]  # MWのサンプル数=Fのサンプル数に

# 新しいバランスの取れたデータセット
balanced_indices = np.concatenate([mw_sample_indices, f_sample_indices])
X_balanced = X[balanced_indices]
y_balanced = y[balanced_indices]

# データにNaNが含まれているか確認
nan_in_data = np.isnan(X_balanced).any()

# NaNがある場合、それを処理
if nan_in_data:
    # NaN値を列の平均で置き換える例
    from sklearn.impute import SimpleImputer
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    X_balanced = imputer.fit_transform(X_balanced)

    # あるいは、NaNを含む行を削除する方法もありますが、これによりデータ量が減少する可能性があります。


# 既存のコード: バランスの取れたデータで訓練セットとテストセットに分割
#X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

# 性能指標を定義
scoring = {'accuracy': 'accuracy',
           'precision': 'precision',
           'recall': make_scorer(recall_score),
           'specificity': make_scorer(lambda y_true, y_pred: confusion_matrix(y_true, y_pred).ravel()[0] / (confusion_matrix(y_true, y_pred).ravel()[0] + confusion_matrix(y_true, y_pred).ravel()[1]))}

# シャッフルされた交差検証を設定
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# RBFカーネルでSVMを実行するための設定
#svm = SVC(kernel='rbf', C=0.00001, gamma='auto')

# 交差検証でのパフォーマンスを評価
#scores = cross_validate(svm, X_balanced, y_balanced, scoring=scoring, cv=10, return_train_score=True)

# 交差検証の実行(多項式カーネル)
#scores = cross_validate(SVC(kernel='poly', degree=3, C=1.0, gamma='scale'), X_balanced, y_balanced, scoring=scoring, cv=10, return_train_score=True)

# ランダムフォレストの設定
#rf = RandomForestClassifier(n_estimators=100, random_state=42)

#rf = RandomForestClassifier(n_estimators=50, max_depth=None, min_samples_split=2, min_samples_leaf=1, random_state=42)
#rf = RandomForestClassifier(n_estimators=39, max_depth=7, min_samples_split=7, min_samples_leaf=9, random_state=42)

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 30)
    max_depth = trial.suggest_categorical('max_depth', [None, 3, 4, 5, 6, 7, 8, 9, 10])
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
                                random_state=42)
    scores = cross_val_score(rf, X_balanced, y_balanced, cv=5, scoring='accuracy')
    return scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print("Best parameters:", study.best_params)
print("Best accuracy:", study.best_value)

# Optunaで得られたベストパラメータ
best_params = study.best_params

# ベストパラメータを使ってランダムフォレストモデルを作成
rf = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                            max_depth=best_params['max_depth'],
                            min_samples_split=best_params['min_samples_split'],
                            min_samples_leaf=best_params['min_samples_leaf'],
                            random_state=42)

# 交差検証でのパフォーマンスを評価
scores = cross_validate(rf, X_balanced, y_balanced, scoring=scoring, cv=10, return_train_score=True)

# 各フォールドでのテストデータの精度を表示
for i in range(10):
    print(f"Fold {i+1}")
    print(f"Train Accuracy: {scores['train_accuracy'][i] * 100:.2f}%")
    print(f"Test Accuracy: {scores['test_accuracy'][i] * 100:.2f}%")

# 平均精度と他の性能指標の計算
train_accuracy = np.mean(scores['train_accuracy']) * 100
test_accuracy = np.mean(scores['test_accuracy']) * 100
precision = np.mean(scores['test_precision']) * 100
recall = np.mean(scores['test_recall']) * 100
specificity = np.mean(scores['test_specificity']) * 100

#結果の出力
print(f"\nTrain Accuracy: {train_accuracy:.2f}%")
print(f"Test Accuracy: {test_accuracy:.2f}%")
print(f"Precision: {precision:.2f}%")
print(f"Recall: {recall:.2f}%")
print(f"Specificity: {specificity:.2f}%")

In [ ]:
#データ出力
print(data.shape)
print(feature.shape)
s = np.std(data[:6, :2096], axis=1)
print(s)
print(X.shape)
print(X_balanced.shape)
print(y_balanced.shape)

In [ ]:
#optunaを使ったハイパーパラメータ最適化(Random Forest)
!pip install optuna
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    max_depth = trial.suggest_int('max_depth', 2, 32, log=True)

    rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
    scores = cross_val_score(rf, X_balanced, y_balanced, cv=5, scoring='accuracy')
    return scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Best parameters:", study.best_params)
print("Best accuracy:", study.best_value)

In [ ]:
#Grid searchによるハイパーパラメータ最適化(Random Forest)
from sklearn.model_selection import GridSearchCV

# パラメータグリッドの設定
param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# グリッドサーチの設定
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid, cv=10, scoring='accuracy')

# グリッドサーチの実行
grid_search.fit(X_balanced, y_balanced)

# 最適なパラメータとスコアの表示
print("Best parameters:", grid_search.best_params_)
print("Best recall:", grid_search.best_score_)


In [ ]:
#Grid searchによるハイパーパラメータ最適化(SVM)
from sklearn.metrics import precision_score, recall_score

# Grid Search用のパラメータグリッドを設定
param_grid = {
    'kernel': ['rbf', 'poly', 'sigmoid'],
    'C': [0.000000001, 0.000001,  0.00001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1],
    'gamma': ['scale', 'auto', 0.01, 0.05, 0.1, 0.5, 1.0, 1.5, 4, 5]
}
"""
def custom_score(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    return min(precision, recall)  # PrecisionとRecallの小さい方をスコアとして返す

# GridSearchCVでの使用例
grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=10, scoring=make_scorer(custom_score))
"""
# RBFカーネル用のGrid Searchの実行
grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=10, scoring='recall')
grid_search.fit(X_balanced, y_balanced)

# 最適なパラメータの表示
print("Best parameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)

Best parameters: {'C': 1e-09, 'gamma': 1.0}
Best accuracy: 0.5185214739851837


In [ ]:
#CNNを使ったMW識別

import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from numpy.core.multiarray import asarray
import csv
import pandas as pd
import numpy as np
from numpy import loadtxt
from torch.utils.data.dataset import Dataset
from PIL import Image
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import f1_score
!pip install scikit-optimize
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import cross_val_score
import torch.nn.init as init
import random
!pip install optuna
import optuna
import torch.optim as optim
#正解率などを計算
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
#!pip install optuna[visualization]
#import torchmetrics
#91.78% accuracy, 92.84% sensitivity(recall), and 90.73% specificity.

#drop0.5814981354327653 lr 0.0003952902573873088

#ハイパーパラメータ
device = torch.device("cuda")
num_epochs = 20
num_classes = 2
batch_size = 13
learning_rate = 0.001

def time_shift(data, shift_max = 1000):
  shift = random.randint(-shift_max, shift_max)
  return np.roll(data, shift, axis = 1)

def add_noise(data, noise_level=0.01):
  noise = np.random.normal(0, noise_level, data.shape)
  return data + noise

def apply_scaling(data, scale_min=0.8, scale_max=1.2):
    scales = np.random.uniform(scale_min, scale_max, data.shape[0])
    scaled_data = data * scales[:, np.newaxis]
    return scaled_data

class CustomDatasetFromCSV(Dataset):
  def __init__(self, csv_path, a, b, c, d, length, fold):
    self.fold = fold
    self.transforms = transforms
    self.to_tensor = transforms.ToTensor()
    self.data_info = pd.read_csv(csv_path)
    if c == 0:
      self.image_arr = np.asarray(self.data_info.iloc[a:b, 0])
      self.label_arr = np.asarray(self.data_info.iloc[a:b, 1])
    else:
      self.image_arr = np.concatenate((np.asarray(self.data_info.iloc[a:b, 0]), np.asarray(self.data_info.iloc[c:d, 0])))
      self.label_arr = np.concatenate((np.asarray(self.data_info.iloc[a:b, 1]), np.asarray(self.data_info.iloc[c:d, 1])))

    #print(f"Image Array for fold {self.fold}: ", self.image_arr)
    print(f"Fold: {self.fold}")

    self.image_len = length
    #print("index", self.image_len)

  def __getitem__(self, index):
    single_image_name = self.image_arr[index]

    # ここでファイル名を出力（デバッグ用）
    #print(f"Reading data from {single_image_name}.txt")

    lines_img = loadtxt(single_image_name + ".txt", delimiter=",", unpack=False)

    # Debugging print statements
    #print("Shape of lines_img before reshape:", lines_img.shape)

    #データ拡張の適用
    #lines_img = time_shift(lines_img) #時間的シフト
    #lines_img = add_noise(lines_img) #ノイズ追加
    #lines_img = apply_scaling(lines_img)

    pad_size = 10240 - lines_img.shape[1]
    padded_lines_img = np.pad(lines_img, [(0,0), (0, pad_size)], "constant")
    padded_lines_img = np.reshape(padded_lines_img, (1, 64, 10240))
    img_as_tensor = torch.FloatTensor(padded_lines_img)

    #lines_img = np.reshape(lines_img, (1,64,10240))

    single_image_label = self.label_arr[index]
    single_image_label = torch.LongTensor([single_image_label])
    #single_image_label = torch.tensor([single_image_label], dtype = torch.long)
    return (img_as_tensor, single_image_label)

  def __len__(self):
    return self.image_len

"""
class CustomDatasetFromCSV(Dataset):
    def __init__(self, csv_path, a, b, c, d, length, fold):
        self.fold = fold
        self.data_info = pd.read_csv(csv_path)
        self.preloaded_data = []
        self.label_arr = []

        # データのパスを取得
        if c == 0:
            image_paths = np.asarray(self.data_info.iloc[a:b, 0])
            self.label_arr = np.asarray(self.data_info.iloc[a:b, 1])
        else:
            image_paths = np.concatenate((np.asarray(self.data_info.iloc[a:b, 0]), np.asarray(self.data_info.iloc[c:d, 0])))
            self.label_arr = np.concatenate((np.asarray(self.data_info.iloc[a:b, 1]), np.asarray(self.data_info.iloc[c:d, 1])))

        # すべてのデータを一度にメモリにロード
        for img_path in image_paths:
            lines_img = loadtxt(img_path + ".txt", delimiter=",", unpack=False)
            lines_img = time_shift(lines_img)
            lines_img = add_noise(lines_img)
            pad_size = 10240 - lines_img.shape[1]
            padded_lines_img = np.pad(lines_img, [(0,0), (0, pad_size)], "constant")
            padded_lines_img = np.reshape(padded_lines_img, (1, 64, 10240))
            self.preloaded_data.append(padded_lines_img)

        self.preloaded_data = np.array(self.preloaded_data)

    def __getitem__(self, index):
        # メモリにロードされたデータを直接使用
        img_as_tensor = torch.FloatTensor(self.preloaded_data[index])
        single_image_label = torch.tensor([self.label_arr[index]], dtype=torch.long)
        return (img_as_tensor, single_image_label)

    def __len__(self):
        return len(self.preloaded_data)
"""

#CNNモデルの定義
class ConvNet(nn.Module):
  def __init__(self, num_classes):
    super(ConvNet, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, (1, 11)),
        nn.Conv2d(32, 32, (64,1)),
        nn.BatchNorm2d(32),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(1,2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 20, (1, 10)),
        nn.BatchNorm2d(20),
        nn.ReLU(True),
        nn.MaxPool2d(1, 4)
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(20, 20, (1, 10)),
        nn.BatchNorm2d(20),
        nn.ReLU(True),
        nn.MaxPool2d(1,4)
    )
    self.layer4 = nn.Sequential(
        nn.Conv2d(20, 20, (1, 11)),
        nn.BatchNorm2d(20),
        nn.ReLU(True),
        nn.MaxPool2d(1,3)
    )
    self.drop_out = nn.Dropout(0.5)
    self.fc1 = nn.Linear(2060, 100)
    self.fc2 = nn.Linear(100, 2)
    #self.fc3 = nn.Linear(50, 2)
    self._initialize_weights()

  def _initialize_weights(self):
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            init.constant_(m.bias, 0)
      elif isinstance(m, nn.Linear):
        init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        init.constant_(m.bias, 0)
    pass

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = out.view(out.size(0), -1)
    #print(f"Before entering fc1, shape of out: {out.shape}")
    out = self.drop_out(out)
    #out = F.relu(self.fc1(out))
    out = self.fc1(out)
    out = self.fc2(out)
    #out = F.relu(self.fc2(out))
    #out = self.drop_out(out)
    #out = self.fc3(out)
    return out

"""
#optuna用のデータロード
optuna_train_dataset = CustomDatasetFromCSV("/content/A.csv", 10, 50, 0, 0, 30, 1)
train_loader = torch.utils.data.DataLoader(optuna_train_dataset, batch_size=5, shuffle=True, num_workers=4)

optuna_val_dataset = CustomDatasetFromCSV("/content/A.csv", 60, 70, 0, 0, 10, 1)
val_loader = torch.utils.data.DataLoader(optuna_val_dataset, batch_size=5, shuffle=True, num_workers=4)

#optunaによるハイパーパラメータ最適化
def objective(trial):
  dropout_rate = trial.suggest_float("dropout_rate", 0.3, 0.7)
  lr = trial.suggest_loguniform("lr", 1e-6, 1e-2)

  model = ConvNet(num_classes, dropout_rate).to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr = lr, weight_decay=1e-5)

  #モデルの訓練+val
  num_epochs=10
  for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
      images = images.to(device)
      labels = labels.to(device)

      outputs = model(images)
      #print("訓練用の正解ラベル = ", labels)
      labels = labels.view(-1)
      loss = criterion(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    #検証ループ
    val_accuracy = calculate_val_accuracy(model, val_loader)
    print(f'Epoch {epoch}: Validation Accuracy = {val_accuracy}%')

    #Pruner
    try:
      trial.report(val_accuracy, epoch)
      if trial.should_prune():
          raise optuna.exceptions.TrialPruned()
    except optuna.exceptions.TrialPruned:
      print(f"Trial {trial.number} early stopped at epoch {epoch}.")
      raise
  return val_accuracy

#ハイパーパラメータ最適化のval精度の計算
def calculate_val_accuracy(model, val_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs.data, 1)
            labels = labels.squeeze()
            print(f"Predicted: {predicted}")
            print(f"Labels: {labels}")

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            print(f"Total: {total}, Correct: {correct}")
    val_accuracy = 100 * correct / total
    return val_accuracy

#Pruner
pruner = optuna.pruners.MedianPruner(
    n_startup_trials=5,  # 最初の2トライアルはプルーニングしない
    n_warmup_steps=2,    # 最初からプルーニングのチェックを開始
    interval_steps=1     # 1エポックごとにプルーニングのチェックを行う
)

#optuna実行
study = optuna.create_study(direction="maximize", pruner = pruner)
study.optimize(objective, n_trials=20)
#進捗の可視化
#optuna.visualization.plot_optimization_history(study)
#最高のハイパーパラメータを設定
best_dropout = study.best_params["dropout_rate"]
best_lr = study.best_params["lr"]
"""

#10-fold交差検証によるモデルの訓練とテスト
ns = 14
train_accuracies = []
test_accuracies = []
# 各指標の合計値を初期化
total_accuracy = 0
total_precision = 0
total_recall = 0
total_f1 = 0
total_specificity = 0
for fold in range(1, 11):
  if fold == 1:
    a1_train = 14
    a2_train = 144
    a1_test = 0
    a2_test = 14
    custom_mnist_from_csv_train = CustomDatasetFromCSV("/content/A.csv", a1_train, a2_train, 0, 0, 130, fold)
    custom_mnist_from_csv_test = CustomDatasetFromCSV("/content/A.csv", a1_test, a2_test, 0, 0, 14, fold)
  elif fold == 10:
    a1_train = 0
    a2_train = 130
    a1_test = 130
    a2_test = 144
    custom_mnist_from_csv_train = CustomDatasetFromCSV("/content/A.csv", a1_train, a2_train, 0, 0, 130, fold)
    custom_mnist_from_csv_test = CustomDatasetFromCSV("/content/A.csv", a1_test, a2_test, 0, 0, 14, fold)
  else:
    a1_train = 0
    a2_train = (fold-2)*ns
    b1_train = (fold-1)*ns
    b2_train = 144
    a1_test = a2_train
    a2_test = b1_train
    custom_mnist_from_csv_train = CustomDatasetFromCSV("/content/A.csv", a1_train, a2_train, b1_train, b2_train, 130, fold)
    custom_mnist_from_csv_test = CustomDatasetFromCSV("/content/A.csv", a1_test, a2_test, 0, 0, 14, fold)

  #訓練データをロードする(13から10へ)
  dataset = torch.utils.data.DataLoader(dataset = custom_mnist_from_csv_train, batch_size=10, shuffle=True, num_workers=8)
  #モデルを設定
  model = ConvNet(num_classes).to(device)
  #model = ConvNet(num_classes, best_dropout).to(device) #optuna
  #重み初期化
  #model._initialize_weights()
  #モデルをCudaに移す
  model = model.cuda()

  #損失関数設定
  criterion = nn.CrossEntropyLoss()
  #optimizer設定
  optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay=1e-5)
  #optimizer = torch.optim.Adam(model.parameters(), lr = best_lr, weight_decay=1e-5)

  total_step = len(dataset)

  sumtrain = 0
  correct = 0
  loss = 0
  best_accuracy = 0.0
  patience = 4
  counter = 0
  for epoch in range(num_epochs):
    #print("train_accuracy = ", sumtrain)
    #print("loss = ", loss)

    total_train_data = 0
    sumtrain = 0

    for i, (images, labels) in enumerate(dataset):
      images = images.to(device)
      labels = labels.to(device)
      total_train_data += len(labels)

      outputs = model(images)
      labels = labels.view(-1)
      loss = criterion(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      outputs = torch.cuda.FloatTensor(outputs)
      #outputs = torch.tensor(outputs, dtype=torch.float, device='cuda')

      correct = 0
      for j in range(len(outputs)):
        if (outputs[j, 0] < outputs[j, 1]) and torch.eq(labels[j], 1):
          correct += 1
        elif (outputs[j, 0] > outputs[j, 1]) and torch.eq(labels[j], 0):
          correct += 1

      sumtrain += correct
    #エポックごとの訓練精度
    train_accuracy = (sumtrain / total_train_data) * 100
    print(f"Epoch {epoch+1}, Train Accuracy: {train_accuracy: .2f}%")

    #Early Stopping
    if train_accuracy > best_accuracy:
      best_accuracy = train_accuracy
      counter = 0
    else:
      counter += 1
    if counter >= patience:
      print("Early stopping")
      break
  #各foldごとの訓練精度
  train_accuracies.append(best_accuracy)

  #テストデータロード。num_workersで並列処理をして高速化
  dataset = torch.utils.data.DataLoader(dataset=custom_mnist_from_csv_test, batch_size=7, shuffle=True, num_workers=8)
#9から７へ
  #f1 score用
  P_labels = []
  T_labels = []
  #テスト用のバッチ
  for i, (images, labels) in enumerate(dataset):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1)
    correct += (predicted==labels).sum().item()
    P_labels.extend(predicted.cpu().numpy())
    T_labels.extend(labels.cpu().numpy())

  # 追加する指標の計算
  total_accuracy += accuracy_score(T_labels, P_labels) * 100
  total_precision += precision_score(T_labels, P_labels) * 100
  total_recall += recall_score(T_labels, P_labels) * 100
  total_f1 += f1_score(T_labels, P_labels)
  tn, fp, fn, tp = confusion_matrix(T_labels, P_labels).ravel()
  total_specificity += tn / (tn + fp) * 100


# 最終的な平均値を計算
average_train_accuracy = sum(train_accuracies) / len(train_accuracies)
average_accuracy = total_accuracy / 10
average_precision = total_precision / 10
average_recall = total_recall / 10
average_f1 = total_f1 / 10
average_specificity = total_specificity / 10

# 結果の出力
print(f"Final Train Accuracy = {average_train_accuracy: .2f}%")
print(f"Accuracy = {average_accuracy: .2f}%")
print(f"Average Recall = {average_recall: .2f}%")
#print(f"Average Precision: = {average_precision: .2f}%")
#print("Average F1 Score:", average_f1)
#print("Average Specificity:", average_specificity)

#出力
#print("Length of train_accuracies", len(train_accuracies))
#print(f"Final Train Accuracy = {average_train_accuracy: .2f}%")

In [ ]:
print(f"Final Train Accuracy = {average_train_accuracy: .2f}%")
print(f"Accuracy = {average_accuracy: .2f}%")
print(f"Average Recall = {average_recall: .2f}%")
print(f"Average Specificity: = {average_specificity: .2f}%")

print(f"Average Precision: = {average_precision: .2f}%")
print("Average F1 Score:", average_f1)